In [4]:
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import geopandas as gpd
import rioxarray as rio
import xarray as xr
import numpy as np

from matplotlib.colors import LinearSegmentedColormap
from shapely.geometry import mapping
from src import geoaxes

plt.style.use('dark_background')

- *Escala de cores utilizada pelo ONS para representar chuva*

In [ ]:
cm_data_ = [[1.0, 1.0, 1.0],
           [0.8823529411764706, 1.0, 1.0],
           [0.7058823529411765, 0.9411764705882353, 0.9803921568627451],
           [0.5882352941176471, 0.8235294117647058, 0.9803921568627451],
           [0.1568627450980392, 0.5098039215686274, 0.9411764705882353],
           [0.0784313725490196, 0.39215686274509803, 0.8235294117647058],
           [0.403921568627451, 0.996078431372549, 0.5215686274509804],
           [0.09411764705882353, 0.8431372549019608, 0.023529411764705882],
           [0.11764705882352941, 0.7058823529411765, 0.11764705882352941],
           [1.0, 0.9098039215686274, 0.47058823529411764],
           [1.0, 0.7529411764705882, 0.23529411764705882],
           [1.0, 0.3764705882352941, 0.0],
           [0.8823529411764706, 0.0784313725490196, 0.0],
           [0.984313725490196, 0.3686274509803922, 0.4196078431372549],
           [0.6666666666666666, 0.6666666666666666, 0.6666666666666666]]

ons_cmap = LinearSegmentedColormap.from_list('ons', cm_data_)
ons_cmap_r = LinearSegmentedColormap.from_list('ons', cm_data_[::-1])

- *Função para recortar dados a partir de um shapefile*

In [ ]:
def mask_data(ds: xr.Dataset, mask: str) -> xr.Dataset:
    """
    Mask data within a geographical extension directly from a shapefile.

    Parameters
    ----------
        ds : ``xarray.Dataset``
            Two-dimensional (2D) array database to which to apply mask.
            
        mask : str, path object or file-like object
            Shapefile to mask dataset.

    Returns
    -------
        ``xarray.Dataset``
            Masked data.
    """
    # GeoDataFrame from a shapefile 
    shape = gpd.read_file(mask)
    # Set the spatial dimensions of the dataset
    ds.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)
    # Write the CRS to the dataset in a CF compliant manner
    ds.rio.write_crs("epsg:4326", inplace=True)
    # Clip using a GeoDataFrame
    return ds.rio.clip(shape.geometry.apply(mapping), 
                       shape.crs, drop=False)

- *Tratamento de dados*

In [ ]:
# Abre o dataset a partir de um arquivo netCDF
# MUDE O CAMINHO DO ARQUIVO DE ACORDO COM SUA PASTA
# VÁ ATÉ O ARQUIVO E CLIQUE BOTÃO DIREITO "COPIAR O CAMINHO"
# ABRA ASPAS E COLE O CAMINHO
ds_precip = xr.open_dataset(r'D:\UFF - Projects\Nivelamento LAMMOC\data\precip_79_22.nc')
# Conversão de unidades (metros para milímetro de chuva)
ds_precip = ds_precip * 1000

# ------------ CLIMATOLOGIA ------------ #

# Período de 30 anos (1991-2020)
clim_period = ['1991-01-01', '2020-12-01']
# Recorta dados para o período selecionado
ds_clim = ds_precip.sel(time=slice(clim_period[0], clim_period[1]))
# Calcular climatologia mensal
ds_clim = ds_clim.groupby('time.month').mean()

# ------------ OBSERVADO ------------#

# Calcular médias mensais de 2022
obs_period = ['2022-01-01', '2022-12-01']
# Recorta dados para o período selecionado
ds_obs = ds_precip.sel(time=slice(obs_period[0], obs_period[1]))
# Calcular climatologia mensal
ds_obs = ds_obs.groupby('time.month').mean()

# ------------ ANOMALIA ------------#

# Calcular anomalias mensais (anomalia = desvio da média climatológica)
# Calcular em valores percentuais (%)
ds_anomaly = (ds_obs - ds_clim) # em mm
ds_anomaly_percentage = ((ds_obs/ds_clim)-1)*100 # em %

# Arquivo shapefile do território brasileiro
# MUDE O CAMINHO DO ARQUIVO DE ACORDO COM SUA PASTA
# VÁ ATÉ O ARQUIVO E CLIQUE BOTÃO DIREITO "COPIAR O CAMINHO"
# ABRA ASPAS E COLE O CAMINHO
shapefile_br = r'D:\UFF - Projects\Nivelamento LAMMOC\data\UFEBRASIL.shp'
# Recortar dados a partir de shapefile
ds_obs = mask_data(ds=ds_obs, mask=shapefile_br)
ds_clim = mask_data(ds=ds_clim, mask=shapefile_br)
ds_anomaly= mask_data(ds=ds_anomaly, mask=shapefile_br)
ds_anomaly_percentage = mask_data(ds=ds_anomaly_percentage, mask=shapefile_br)

- *Visualização de dados (mapa)*

In [ ]:
# Sequência com os 12 meses
meses = np.arange(1, 13, 1)

for mes in meses:
    # Cria figura e as áreas de plotagem
    # Serão 3 plots : observado x climatologia x anomalia
    fig, (ax1, ax2, ax3) = plt.subplots(
                                subplot_kw=dict(projection=ccrs.PlateCarree()), 
                                nrows=1, 
                                ncols=3,
                                figsize=(30, 10)
                            )
    # Adiciona título à figura
    fig.suptitle(
        f'Análise espacial da chuva para o mês {mes}', 
        fontsize=25, 
        weight='bold'
    )
    # Ajusta figura para agrupar os diferentes plots
    fig.subplots_adjust()

    # -------------- CHUVA OBSERVADA (2022) -------------- #

    # Plota mapa de coloração 
    # Este mapa representa a chuva observada em 2022
    observed = ax1.contourf(
                    ds_obs['longitude'].values,
                    ds_obs['latitude'].values,
                    ds_obs['tp'].sel(month=mes),
                    np.arange(0, 16, 1),
                    cmap=ons_cmap,
                    extend='both'  
                )
    # Adiciona escala de cor
    cbar_1 = plt.colorbar(
                    mappable=observed,
                    orientation='horizontal',
                    ax=ax1,
                )
    # Propriedades da escala de cor
    cbar_1.ax.tick_params(labelsize=20)
    cbar_1.set_label('mm', fontsize=20, labelpad=20)

    # Dá titulo ao mapa
    ax1.set_title('Chuva observada', fontsize=20)

    # -------------- CHUVA CLIMATOLÓGICA (1991-2020) -------------- #

    # Plota mapa de coloração 
    # Este mapa representa a chuva climatológica para o mês
    climatology = ax2.contourf(
                        ds_clim['longitude'].values,
                        ds_clim['latitude'].values,
                        ds_clim['tp'].sel(month=mes),
                        np.arange(0, 16, 1),
                        cmap=ons_cmap,
                        extend='both'  
                    )
    # Adiciona escala de cor
    cbar_2 = plt.colorbar(
                    mappable=climatology,
                    orientation='horizontal',
                    ax=ax2,
                )
    # Propriedades da escala de cor
    cbar_2.ax.tick_params(labelsize=20)
    cbar_2.set_label('mm', fontsize=20, labelpad=20)

    # Dá titulo ao mapa
    ax2.set_title('Chuva climatológica', fontsize=20)

    # -------------- ANOMALIA OBSERVADA DA CHUVA -------------- #

    # Plota mapa de coloração 
    # Este mapa representa a anomalia de chuva observada no mês
    anomaly = ax3.contourf(
                    ds_anomaly_percentage['longitude'].values,
                    ds_anomaly_percentage['latitude'].values,
                    ds_anomaly_percentage['tp'].sel(month=mes),
                    np.arange(-100, 110, 10),
                    cmap='coolwarm_r',
                    extend='both' 
                )
    # Adiciona escala de cor
    cbar_3 = plt.colorbar(
                    mappable=anomaly,
                    orientation='horizontal',
                    ax=ax3,
                )
    # Propriedades da escala de cor
    cbar_3.ax.tick_params(labelsize=20)
    cbar_3.set_label('%', fontsize=20, labelpad=20)

    # Dá titulo ao mapa
    ax3.set_title('Anomalia observada', fontsize=20)

    # Personaliza mapas
    for area_plotagem in [ax1, ax2, ax3]:
        geoaxes.GeoAxes(ax=area_plotagem).geosettings(
                                            extent=[-80, -30, 10, -35],
                                            shapefile=shapefile_br,
                                            ylocator=np.arange(-180, 180, 10)
                                        )
    # Salvar figura como arquivo .png na pasta "images"
    if mes < 10: 
        plt.savefig(f'./images/analise_espacial_chuva_2022_0{mes}')
    else:
        plt.savefig(f'./images/analise_espacial_chuva_2022_{mes}')